In [6]:
%load_ext autoreload
%autoreload 2

import os,sys
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))
sys.path.append(os.path.abspath("../training"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
os.environ["EXP_NAME"] = "movie_recom"
os.environ["MAX_RATING"] = "5"
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:8081"
os.environ["EPOCH"] = "2"

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

In [8]:
from training.xgb import main, test_xgb_model
from movie_recommender.workflow import read_parquet_from_s3, connect_minio,connect_mlflow

DB_MINIO_BUCKET = "trainingbucket"
connect_minio()
connect_mlflow()

Available Buckets [Bucket('mlflowbucket'), Bucket('trainingbucket')]


In [9]:
ratings = read_parquet_from_s3(DB_MINIO_BUCKET, "ratings.parquet")
movies = read_parquet_from_s3(DB_MINIO_BUCKET, "movies.parquet")
ratings = ratings.merge(movies, on="movie_id")
ratings.rating *= 5

In [10]:
booster = main(ratings,
               n_trials=1)

[I 2025-09-20 10:06:41,123] A new study created in memory with name: no-name-e289421d-b126-44db-8679-b3831ecc099d


Running CV...


  0%|          | 0/1 [00:00<?, ?it/s]

[I 2025-09-20 10:06:54,480] Trial 0 finished with value: 0.1415920460972587 and parameters: {'objective': 'reg:squarederror', 'device': 'cuda', 'random_state': 0, 'eval_metric': 'mae', 'booster': 'gbtree', 'max_depth': 6, 'learning_rate': 0.011892899714166807, 'subsample': 0.8981469364237483, 'colsample_bytree': 0.8583054839413833, 'gamma': 0.5573575579925505, 'lambda': 3.330146719411467, 'alpha': 0.0303607028072268}. Best is trial 0 with value: 0.1415920460972587.
Found best params: {'objective': 'reg:squarederror', 'device': 'cuda', 'random_state': 0, 'eval_metric': 'mae', 'booster': 'gbtree', 'max_depth': 6, 'learning_rate': 0.011892899714166807, 'subsample': 0.8981469364237483, 'colsample_bytree': 0.8583054839413833, 'gamma': 0.5573575579925505, 'lambda': 3.330146719411467, 'alpha': 0.0303607028072268}
[0]	train-mae:0.16178	val-mae:0.15802
[619]	train-mae:0.08837	val-mae:0.14106


[INFO] - Run id: 26161cc8ad064fc18105ecb7a51da4ce


Found most impactful features: ['movie_avg_rating', 'mean_Drama', 'mean_Adventure', 'mean_Action', 'mean_Thriller', 'mean_Comedy', 'total_Romance', 'user_avg_rating', 'highly rated genre_Adventure', 'highly rated genre_Drama', 'most rated genre_Adventure']

Training XGBMR...
[0]	train-mae:0.16221	val-mae:0.15822
[500]	train-mae:0.09047	val-mae:0.14060
[625]	train-mae:0.09045	val-mae:0.14061


2025/09/20 10:07:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/20 10:07:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/phantom/miniconda3/envs/cudatorch/lib/python3.12/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [10:07:03] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
[INFO] - Logging artifacts.
[INFO] - Done training.


Logging folder /tmp/tmp0lm3vk66
🏃 View run brawny-grouse-692 at: http://localhost:8081/#/experiments/1/runs/26161cc8ad064fc18105ecb7a51da4ce
🧪 View experiment at: http://localhost:8081/#/experiments/1


Registered model 'xgb' already exists. Creating a new version of this model...
2025/09/20 10:07:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgb, version 7
Created version '7' of model 'xgb'.


metrics {'train-mae': 0.10580228301630404, 'val-mae': 0.13744218488103954, 'stopped_iteration': 517.0, 'best_iteration': 18.0}
Champion 'val-mae': 0.1374, Latest 'val-mae': 0.1380
No new champion for 'xgb'


In [11]:
test_xgb_model()

XGB loaded on device: cuda
Getting artifacts, run_id='26161cc8ad064fc18105ecb7a51da4ce', artifact_path='resources'


/home/phantom/git/n2ak/movie_recommender/backend/movie_recommender/workflow.py:114: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  return _mlflowClient.get_latest_versions(registered_name)[0].run_id


[INFO] - Loaded champion model, registered_name='xgb'
